In [1]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

nltk.download('stopwords')

file_path = "spam.csv"  
df = pd.read_csv(file_path, encoding='latin-1')

df = df[['v1', 'v2']] 
df.columns = ['label', 'message']

df['label'] = df['label'].map({'spam': 1, 'ham': 0})

def clean_text(text):
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", "", text)  
    words = text.split()
    words = [word for word in words if word not in stopwords.words('english')] 
    return " ".join(words)

df['clean_message'] = df['message'].apply(clean_text)

X_train, X_test, y_train, y_test = train_test_split(df['clean_message'], df['label'], test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

models = {
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(),
    "SVM": SVC(kernel='linear')
}

for name, model in models.items():
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_test_tfidf)
    print(f"\n{name} Results:")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))

def predict_sms():
    user_message = input("\nEnter an SMS to check if it's spam or not: ")
    clean_input = clean_text(user_message)
    input_tfidf = vectorizer.transform([clean_input])
    
    predictions = {name: model.predict(input_tfidf)[0] for name, model in models.items()}
    
    for name, prediction in predictions.items():
        result = "SPAM" if prediction == 1 else "HAM (Legitimate)"
        print(f"{name} Prediction: {result}")

predict_sms()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\benaz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Naive Bayes Results:
Accuracy: 0.967713004484305
Classification Report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98       965
           1       1.00      0.76      0.86       150

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.92      1115
weighted avg       0.97      0.97      0.97      1115


Logistic Regression Results:
Accuracy: 0.9497757847533632
Classification Report:
               precision    recall  f1-score   support

           0       0.95      1.00      0.97       965
           1       0.96      0.65      0.78       150

    accuracy                           0.95      1115
   macro avg       0.95      0.82      0.87      1115
weighted avg       0.95      0.95      0.95      1115


SVM Results:
Accuracy: 0.979372197309417
Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       965
           1  